In [1]:
import bs4
import requests
from fake_headers import Headers

In [2]:
# Создаем header 
header = Headers(
        browser="chrome",  # Generate only Chrome UA
        os="win",  # Generate ony Windows platform
        headers=True  # generate misc headers
    )


my_header = header.generate()

print(my_header)

{'Accept': '*/*', 'Connection': 'keep-alive', 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36', 'Accept-Encoding': 'gzip, deflate, br', 'Upgrade-Insecure-Requests': '1', 'Referer': 'https://google.com'}


In [3]:
url = 'https://habr.com/ru/all/'

In [4]:
response = requests.get(url, headers=my_header)

In [5]:
response

<Response [200]>

In [6]:
text = response.text

In [7]:
soup = bs4.BeautifulSoup(text, features='html.parser')

In [9]:
def get_articles(url, header=my_header):
    '''
    получает список articles
    '''
    response = requests.get(url, headers=my_header)
    text = response.text
    soup = bs4.BeautifulSoup(text, features='html.parser')
    articles = soup.find_all('article')
    return articles

In [10]:
def get_hubs(articles):
    '''
    получает списки: заголовок статьи, ссылка, дата
    '''
    hubs_list = []
    links_list = []
    dates_list = []
    for article in articles:
        hubs = article.find_all(class_='tm-article-snippet__title tm-article-snippet__title_h2')
        hubs = [hub.text.strip() for hub in hubs]
        hubs_list.extend(hubs)
        links_list.append('https://habr.com' + article.find(class_='tm-article-snippet__title-link').attrs['href'])
        dates_list.append(article.find(class_='tm-article-datetime-published').find('time').attrs['title'])
    return hubs_list, links_list, dates_list
    

In [14]:
#Найдем последнюю страницу
end_page = int(soup.find_all(class_ = 'tm-pagination__page')[-1].text.strip())
hubs = []
links = []
dates = []
for page in range(1, end_page+1):
    url = f'https://habr.com/ru/all/page{page}/'
    articles = get_articles(url, header=my_header)
    h, l, d = get_hubs(articles)
    hubs.extend(h)
    links.extend(l)
    dates.extend(d)

In [15]:
len(hubs) == len(links) == len(dates)

True

In [16]:
# Ключевые слова
keywords = ['дизайн', 'фото', 'web', 'python']

In [17]:
def is_exist_keyword (row, keywords):
    '''
    проверяет, встречается ли ключевое слово в строке
    '''
    for word in keywords:
        if word in row:
            return True
    return False

In [18]:
# находим индексы заголовок статей, в которых встречаются ключевые слова
index = []
for i in range(len(hubs)):
    is_exist = is_exist_keyword(hubs[i], keywords)
    if is_exist:
        index.append(i)

In [19]:
index

[7, 225, 328, 601, 625, 667, 719, 956]

In [20]:
# Вывести в консоль список подходящих статей в формате: <дата> - <заголовок> - <ссылка>.

In [21]:
for i in index:
    print(f'<{dates[i]}> - <{hubs[i]}> - <{links[i]}>')

<2023-02-17, 12:21> - <Проектируем дизайн честного кредитного калькулятора> - <https://habr.com/ru/post/717584/>
<2023-02-14, 09:30> - <Пишем чат-бот для Telegram на Python, используя webhook и минимум внешних библиотек> - <https://habr.com/ru/company/digitalleague/blog/716760/>
<2023-02-12, 14:16> - <Тест дизайн методом Interface — Model — State> - <https://habr.com/ru/post/716382/>
<2023-02-07, 10:01> - <Как стать веб-дизайнером> - <https://habr.com/ru/company/lanit/blog/713708/>
<2023-02-06, 16:31> - <Из дизайнера в инженеры: как спроектировать сложный интерфейс, если никто не знает, что должно получиться в итоге> - <https://habr.com/ru/post/713918/>
<2023-02-06, 07:00> - <Удаленный доступ к IP камерам. Часть 3. HEVC и web> - <https://habr.com/ru/post/715016/>
<2023-02-04, 12:39> - <Комьюнити дизайнеров в продукте> - <https://habr.com/ru/post/714836/>
<2023-01-31, 10:44> - <Зачем России гигаваттный завод по производству кремниевых пластин и фотоэлектрических ячеек?> - <https://habr.

## Дополнительное (необязательное) задание

### Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

In [46]:
def get_article_full_text(url, headers=my_header):
    '''
    возвращает текст статей
    '''
    response = requests.get(url, headers=my_header)
    text = response.text
    soup = bs4.BeautifulSoup(text, features='html.parser')
    
    article_text = soup.find(class_ = 'article-formatted-body article-formatted-body article-formatted-body_version-2')
    if article_text is not None:
        return article_text.text
    article_text = soup.find(id = 'post-content-body').text
    return article_text 
    

In [61]:
url = 'https://habr.com/ru/all/'
response = requests.get(url, headers=my_header)
text = response.text
soup = bs4.BeautifulSoup(text, features='html.parser')
end_page = int(soup.find_all(class_ = 'tm-pagination__page')[-1].text.strip())
hubs = []
links = []
dates = []
full_text = []
is_exist_in_article = False
is_exist_in_article_full_text = False
for page in range(1, end_page+1):
    if page % 10 == 0:
        print(page)
    url = f'https://habr.com/ru/all/page{page}/'
    articles = get_articles(url, header=my_header)
    h, l, d = get_hubs(articles)

    for i in range(len(h)):
        articles_full_text = get_article_full_text(l[i], my_header)
        is_exist_in_article = is_exist_keyword(h[i], keywords)
        is_exist_in_article_full_text = is_exist_keyword(articles_full_text[i], keywords)
        if is_exist_in_article or is_exist_in_article_full_text:
            hubs.append(h[i])
            links.append(l[i])
            dates.append(d[i])
            full_text.append(articles_full_text)
        else:
            is_exist_in_article = False
            is_exist_in_article_full_text = False

10
20
30
40
50


In [62]:
hubs

['Проектируем дизайн честного кредитного калькулятора',
 'Пишем чат-бот для Telegram на Python, используя webhook и минимум внешних библиотек',
 'Тест дизайн методом Interface — Model — State',
 'Как стать веб-дизайнером',
 'Из дизайнера в инженеры: как спроектировать сложный интерфейс, если никто не знает, что должно получиться в итоге',
 'Удаленный доступ к IP камерам. Часть 3. HEVC и web',
 'Комьюнити дизайнеров в продукте',
 'Зачем России гигаваттный завод по производству кремниевых пластин и фотоэлектрических ячеек?']

In [65]:
for i in range(len(hubs)):
    print(f'<{dates[i]}> - <{hubs[i]}> - <{links[i]}>')

<2023-02-17, 12:21> - <Проектируем дизайн честного кредитного калькулятора> - <https://habr.com/ru/post/717584/>
<2023-02-14, 09:30> - <Пишем чат-бот для Telegram на Python, используя webhook и минимум внешних библиотек> - <https://habr.com/ru/company/digitalleague/blog/716760/>
<2023-02-12, 14:16> - <Тест дизайн методом Interface — Model — State> - <https://habr.com/ru/post/716382/>
<2023-02-07, 10:01> - <Как стать веб-дизайнером> - <https://habr.com/ru/company/lanit/blog/713708/>
<2023-02-06, 16:31> - <Из дизайнера в инженеры: как спроектировать сложный интерфейс, если никто не знает, что должно получиться в итоге> - <https://habr.com/ru/post/713918/>
<2023-02-06, 07:00> - <Удаленный доступ к IP камерам. Часть 3. HEVC и web> - <https://habr.com/ru/post/715016/>
<2023-02-04, 12:39> - <Комьюнити дизайнеров в продукте> - <https://habr.com/ru/post/714836/>
<2023-01-31, 10:44> - <Зачем России гигаваттный завод по производству кремниевых пластин и фотоэлектрических ячеек?> - <https://habr.

In [66]:
# hubs - список заголовков, в которых есть ключевые слова
# dates - список дат статей
# links - список ссылок на статьи
# full_text - список полных текстов статей, в которых есть клбчевые слова